In [3]:
import requests
from pydantic import BaseModel, ValidationError, field_validator, TypeAdapter, Field
import json
from typing import Literal
from datetime import datetime
import os

login = os.getenv("login")
password = os.getenv("password")
client_id = os.getenv("client_id")
accounts = os.getenv("accounts")

url_data = f"https://auto.procontext.ru/api/yad/stat/petrovichstat?clientId={client_id}"
login_url = "https://auto.procontext.ru/api/login"
response_login = requests.post(login_url,data={"login": login, "password": password })

api_token = response_login.json()["data"]["api_token"]

body = {
    "accounts":[accounts],
    "date":["2024-01-01","2024-02-02"]
}
body
headers = {'Authorization': f'Bearer {api_token}'}
response = requests.post(url_data,json=body,headers=headers)


# Модель проверки данных
class UniqueProcontext(BaseModel):
  cid: int
  gid: int
  bid: int
  block_type: Literal['SEARCH', 'AD_NETWORK'] = Field(alias='type')
  date: datetime
  device: Literal['mobile', 'desktop', 'tablet']
  val: int
  clicks: int
  sum: float
  trans: int
  shows: int
  cname: str
  gname: str

  @field_validator('cid')
  def valid_cid(cls, value):
    str_value = str(value)
    if (len(str_value)==8):
      return value
    raise ValueError(f"Parametr 'cid' should have lenth of 8, {len(str_value)}")

  @field_validator('gid')
  def valid_gid(cls, value):
    str_value = str(value)
    if (len(str_value)==10) or (value==0):
      return value
    raise ValueError(f"Parametr 'gid' should have lenth of 10, {len(str_value)}")

  @field_validator('bid')
  def valid_bid(cls, value):
    str_value = str(value)
    if (len(str_value)==11) or (value==0):
      return value
    raise ValueError(f"Parametr 'bid' should have lenth of 11, {len(str_value)}")

class TotalResponce(BaseModel):
  data: list[UniqueProcontext]

def ValidateResponce(responce_data):
  exeptions = 'No exceptoins'
  try:
    TotalResponce.parse_obj(responce_data)
  except ValidationError as e:
    exeptions = json.loads(e.json())
  return exeptions


errors = ValidateResponce({'data':response.json()})
errors

'No exceptoins'